In [16]:
import os
import sys
import datetime
import gspread
import pandas as pd
from urllib.parse import quote
from oauth2client.service_account import ServiceAccountCredentials
from sqlalchemy import create_engine
from sqlalchemy import select

## Function ## match result 유효한 데이터 값만 가져오기
def gt_matchRecord(st_data):
    rows = list()
    for x in range(3,30):
        if st_data[x][2] == '-':
            break
        extend_row = list(st_data[x][0:15])
        rows.append(extend_row)
    return rows

def gt_matchSummary(st_data):
    rows = list()
    for x in range(3,30):
        if st_data[x][20] == '':
            break
        else:
            if st_data[x][21] == '':
                pass
            else:
                extend_row = list(st_data[x][20:29])
                rows.append(extend_row)
    return rows    


# Google API 연결 설정 #
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive',]
work_path = os.getcwd()
resource_path = os.path.join(work_path,'resources') # resources 디렉토리 path 설정
json_file_name = os.path.join(resource_path,'quiet-amp-275114-083a1f3b2f00.json')              
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

# DB ORM 연결, 패스워드 특수문자 치환 #
engine = create_engine('postgresql://app_fs:%s@localhost:5432/db_fs' % quote('pok1234@'),echo = True) 
result = engine.execute("select * from tb_sheeturls")
url_data = result.fetchall()
print(url_data)
engine.dispose()

# 구글시트 url 리스트 #
spreadsheet_url = url_data[0][0] # 1경기 URL
match_NumAndDate = url_data[0][1]
match_Num = match_NumAndDate[:3]
format = '%Y-%m-%d'
match_Date = datetime.datetime.strptime(match_NumAndDate[-8:], "%Y%m%d").date()

# 스프레스시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
worksheet = doc.sheet1  ## 시트 선택하기
sheet_data = worksheet.get_all_values()  ## 시트 전체 값을 List of List 로 저장

# 3번째 행을 header로 지정
#header = sheet_data[2][20:22]

#pd_rows = gt_matchResult(sheet_data)
pd_rows = gt_matchSummary(sheet_data)

# Pandas 출력
matchdata = pd.DataFrame(pd_rows, columns= ["team","name","win","draw","lose","득점","실점","goal","assist"])
matchdata.insert(0,'mnum',match_Num)
matchdata.insert(1,'mdate',match_Date)
#matchdata['mnum'] = match_Num
#matchdata['mdate'] = match_Date
print(matchdata)

# DB ORM 연결, 패스워드 특수문자 치환 #
engine = create_engine('postgresql://app_fs:%s@localhost:5432/db_fs' % quote('pok1234@'),echo = True) 
#engine.execute("DROP TABLE IF EXISTS public.tb_test;") # drop table if exists
# data to DBMS Table #
matchdata.to_sql(
         name = 'tb_test',
         con = engine,
         schema = 'public',
         if_exists = 'append',
         index = False
         )

engine.dispose()

2022-03-17 22:41:31,008 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-03-17 22:41:31,008 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-17 22:41:31,009 INFO sqlalchemy.engine.Engine select current_schema()
2022-03-17 22:41:31,009 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-17 22:41:31,010 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-03-17 22:41:31,010 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-17 22:41:31,011 INFO sqlalchemy.engine.Engine select * from tb_sheeturls
2022-03-17 22:41:31,011 INFO sqlalchemy.engine.Engine [raw sql] {}
[('https://docs.google.com/spreadsheets/d/1KQkk_zKzdDziVBGdsh1YuBW4dEL_oZslJ724wRNjLkQ', 'M01_20210704'), ('https://docs.google.com/spreadsheets/d/1mMgwTIRf_1Ohw04lhAsyfvH7iZ681nrDvimUCBWpMbU', 'M02_20210711'), ('https://docs.google.com/spreadsheets/d/1lvKLA_r-FUZvXCpnGPNMGMjyc2tfvk9xL7kB5Dd2ISM', 'M03_20210718'), ('https://docs.google.com/spreadsheets/d/1_6DpvfyeUXqvaalhytHP-5281QyXx18WmPUFQeL48mo', 